## Aim for November 6 publication

## TODO by Monday 28 October
- check authentication
- add option to get https link
- prune imports

## Summary

In this tutorial, we will use NASA's Earthdata Harmony Services to subset and access ICESat-2 data with the `harmony-py` Python library.  Harmony can be used to subset Level-2 and Level-3A ICESat-2 data products.  In this tutorial, we will subset the [ATL03 Global Geolocated Photon Data](https://nsidc.org/data/atl03/versions/6) product as an example.

**What is Harmony?**  [Harmony](https://harmony.earthdata.nasa.gov/) is a service that allows you to customize many NASA datasets, including the ability to subset, reproject and reformat files.  Data can be subsetted for a geographic region, a temporal range and by variable.  Data can be "reprojected" from its native coordinate reference system (CRS) to the coordinate reference system relevant to your analysis.  And data can be reformatted from its native file format to a format that is more relevant for your application.  These services are collectively called _transformation services_.  However, not all services are available for all datasets.  You will learn how to discover which services are available for a given dataset.

Data transformed by Harmony services are staged on NASA Amazon Web Services (AWS) S3 buckets or on user-owned AWS S3 buckets.  Data in NASA S3 buckets are accessed using signed URLs or temporary access credentials.  This data can be downloaded to your local machine or you can access the data directly if you are working in an AWS cloud instance, such as a Jupyter Hub, in AWS `us-west-2`.  

_Add links or provide background to terminology_
_AS: I added a few links above. I think this is sufficient but feel free to add other references if you'd like_

_Stuff for later, maybe_
_AS: I don't think this additional info is needed here but it's a great explanation. I wonder if it ought to be incorporated into NSIDC's cloud access guide: https://nsidc.org/data/user-resources/help-center/nasa-earthdata-cloud-data-access-guide
Data in NASA’s Earthdata Cloud, including the subsetted data processed by Harmony, reside in Amazon Web Services (AWS) Simple Storage Service (S3) buckets. Access is provided via temporary credentials; this free access is limited to requests made within the US West (Oregon) (code: us-west-2) AWS region. While this compute location is required for direct S3 access, all data in Earthdata Cloud are still freely available via download.

## Learning Objectives

In this tutorial you will learn how to:

1. discover Harmony service options for ICESat-2 datasets;
3. use the `harmony-py` library to subset ATL03 granules for a bounding box and time range;
4. download the subsetted ATL03 to your local machine;
5. load the subsetted ATL03 data directly into xarray.

## Prerequisites

This tutorial has been designed to run in an AWS cloud compute instance in AWS region `us-west-2`.  However, if you want run it from your laptop or workstation, everything except Step-4, direct access, should work just fine.  _Also get https link_ _AS: What does that comment reference?_ 

An [Earthdata Login](https://urs.earthdata.nasa.gov) account is required to access data from the NASA Earthdata system. Before requesting a subset of ICESat-2 data, we first need to set up our Earthdata Login authentication, shown below.

## Tutorial

### Step 1: import required packages

We'll be using `earthaccess`, `harmony-py` and `xarray`, along with some Python standard libraries. 

In [18]:
# Harmony services
from harmony import BBox, Client, Collection, Request, LinkType, CapabilitiesRequest 

# Earthdata Login Authentication
import earthaccess 

# Load the data
import xarray as xr

import netrc
import json
import datetime as dt
import s3fs

from IPython.display import display, JSON

### Step 2: start a Harmony client

A Harmony client is the interface to Harmony services and is used to submit requests and retrieve the results of those requests.

To create a Harmony Client we need to supply our Earthdata Login credentials.  We will use `earthaccess.login()` to do this.

:::{note}
`earthaccess.login()` will search for you Earthdata Login credentials stored as environment variables or in a `.netrc` file.  If no environment variables or `.netrc` file is found, we will be prompted for a _username_ and _password_.

See the [`earthaccess` documentation](https://earthaccess.readthedocs.io/en/latest/user_guide/authenticate/) for more details.
:::

We only need to start the client once.  That client can then be used to discover what options are available for a dataset, request subsetting, check on the status of that request and retrieve the results.

_AS ANDY: I removed the comment on the auth line. I'm not sure if earthaccess is even needed if a .netrc exists, as I believe `Client()` is also looking for a netrc. Please update if I am misinterpreting_ 

In [19]:
auth = earthaccess.login()
harmony_client = Client() 

### Step 3: discover service options for ICESAT-2

The first thing we want to know is what service options Harmony has for the ICESat-2 dataset.  We discover service options for a dataset by submitting a `CapabilitiesRequest`.  

A `CapabilitiesRequest` takes a single argument that is either the _Collection ID_ (also known as the _Concept ID_) or the _Short Name_ for the dataset.  A _Concept ID_ is a unique identifier for a dataset.  For ATL03 Version 6, it is C2596864127-NSIDC_CPRD.  For ICESat-2 products, the _Short Name_ is the familiar product name starting with ATL; e.g. ATL03.

Submitting requests to Harmony will follow the same pattern, whether we are discovering capabilities or subsetting data.  We first create a request and then submit that request.

In [20]:
short_name = "ATL03"
capabilities_request = CapabilitiesRequest(short_name=short_name)

We then submit the request using the request method of the Harmony client.

In [21]:
capabilities = harmony_client.submit(capabilities_request)

The result is returned as a [JSON](https://en.wikipedia.org/wiki/JSON) string.  This string is human readable text but the format makes it hard to read the contents.  We can use the `json` library to _decode_ the string and print it in a more reader-friendly format.

:::{warning}
The JSON returned by capabilities can be very long if variable subsetting is an option and data
files contains lots of variables.  However, information about subsetting service options should be in the first few lines of output.
:::

In [22]:
JSON(capabilities)

<IPython.core.display.JSON object>

The JSON response contains a list of transformation options available for each dataset: marked as `true` if available and `false` if not.  A list of Harmony service _endpoints_ and capabilities associated with the transformation options.  And a list of variables in the dataset if variable subsetting is available.

For ATL03, you can see that only subsetting bounding box (`bboxSubset`) and subsetting by Shapefile (`shapeSubset`) are marked `true`.  Variable subsetting (`variableSubset`), concatenation (`concatenate`) and reprojection (`reproject`) are all `false`.   Bounding box and Shapefile subsetting, as well as temporal subsetting, are performed by the Trajectory Subsetter.  This subsetting routine outputs an HDF5 file.

### Step 4: subset ICESat-2 ATL03 file

Now that we know what subsetting options are available for the ATL03 dataset, we can request a subsetted dataset.  

#### Create A Subset Request

we will create a simple request for subset of ATL03 for a bounding box over the northern Colorado Front Range and for the 2020-04-27 to 2020-05-28 period.

![Bounding Box and ATL03 Ground Track for northern Colorado](images/atl03_ground_track_and_bbox.png)

The spatial extent to subset is passed as a `BBox` object that defines the _minimum longitude_, _minimum latitude_, _maximum longitude_ and _maximum latitude_ in decimal degrees.

The temporal range to subset is passed to Harmony as a dictionary with `"start"` and `"stop"` keys, and dates supplied as `datetime` objects.  

Currently, ICESat-2 data only allows spatial and temporal subsetting so we only demonstrate these capabilities.  For further information on constructing a `Request` objects see the [harmony-py](https://harmony-py.readthedocs.io/en/latest/) documentation.

In [23]:
request = Request(
  collection = Collection(id=capabilities["conceptId"]),
  spatial=BBox(-105.5,40,-105,41.),
  temporal={
    'start': dt.datetime(2020, 4, 27),
    'stop': dt.datetime(2020, 5, 28)
  }
)

#### Submit the request

The request is submitted in the same way as the Capabilities request.  This starts the subsetting process, which may take a while depending on the size of the request.  Submitting request returns a Job ID, which is a unique identifier for your request that is used to track the progress of the request and to access the results.

If the request involves a lot of files (more than 300), Harmony will only process the first 300 files.  See section below on [Handling Large Jobs](https://openscapes.2i2c.cloud/user/andypbarrett/lab/workspaces/auto-e/tree/earthdata-cloud-cookbook/tutorials/IS2_Harmony.ipynb#handling-large-requests) on how to work around this restriction. 

_AS: ANDY: The link above is broken for me; it looks like it's pointing to your local jupyter instance. Maybe we could just leave this unlinked for now._

In [24]:
job_id = harmony_client.submit(request)
job_id

'16ef9962-7be4-4434-891a-963189aa7854'

### Step 5: check the status of the request

Subsetting is performed in the cloud.  For small jobs, the subsetting process can be monitored with a progress bar by submitting the `job_id` to the `wait_for_processing` method of the Harmony client.

In [25]:
harmony_client.wait_for_processing(job_id, show_progress=True)

 [ Processing: 100% ] |###################################################| [|]


Once the subsetting has finished, information about the job can be accessed as a JSON file

In [26]:
job_summary = harmony_client.result_json(job_id)
JSON(job_summary)

<IPython.core.display.JSON object>

### Step 6: access the subsetted data

The subsetted files can be accessed by downloading the files to a local machine, such as a laptop or desktop workstation, or by _streaming_ the data.  We will use both access methods in the two examples below.

Results are staged for 30 days in the Harmony s3 bucket.

_AS: ANDY, you had a note to explain the staging location. I updated the sentence above but I don't think this really needs any additional explanation. You have already explained S3 in the initial summary. The key point here is that the data are only available for 30 days._

#### Download a single file

The _download_ method takes a url to a single subsetted file.  The `directory` keyword is used to specify a download path.  The default is the current working directory (`.`).  Setting `overwrite` to False avoids downloading the same file twice.  If you need to download the file again, then set `overwrite=True`.

:::{note}
The `download` and `download_all` method are [_asynchronous_](https://en.wikipedia.org/wiki/Asynchrony_(computer_programming)), so that downloading each file is performed independently.  Once the downloads are completed, the filepaths for the downloaded file are accessed using the `result` method.
:::

In [27]:
url = list(harmony_client.result_urls(job_id))[0]  # Get the data url of the first file
filepath = harmony_client.download(url, directory=".", overwrite=False).result()
filepath

./ATL03_20200427193622_04930702_006_02_subsetted.h5


'./ATL03_20200427193622_04930702_006_02_subsetted.h5'

./ATL03_20200427193622_04930702_006_02_subsetted.h5


#### Download all files

The `download_all` method can use the _job-id_ or the _result-json_, which contains result urls.  

As with `download`, the download directory path on the local machine can be specified with the `directory` keyword.  To save downloading the same file, the `overwrite` keyword can be set to False. 

In [28]:
futures = harmony_client.download_all(job_id, directory=".", overwrite=False)
filelist = [f.result() for f in futures]  # get filepaths
filelist

['./ATL03_20200427193622_04930702_006_02_subsetted.h5']

In [32]:
filelist[0]

'./ATL03_20200427193622_04930702_006_02_subsetted.h5'

#### Load a file into an `xarray` dataset

The simplest way to load ICESat-2 data is to use `xarray`.

:::{note}
You could also load the data into a `geopandas.GeoDataframe`
:::

In [33]:
ds = xr.open_dataset(filelist[0], group="gt1l/heights")
ds

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

#### Direct S3 Access of Harmony Results

If you are working in the AWS `us-west-2` region (the same region as NASA Earthdata Cloud) you can _stream_ the data using direct S3 access.

:::{warning}
You must be running this notebook in the AWS us-west-2 region to run the following code cells.
:::



We need to get the url for the data in the S3 bucket.  We can do this using `result_urls`, as we did for `download` but we set `link_type=LinkType.s3` to specify we want the S3 url.

In [34]:
urls = list(harmony_client.result_urls(job_id, link_type=LinkType.s3))  # result_urls returns a generator possible issue to return list
urls

['s3://harmony-prod-staging/public/16ef9962-7be4-4434-891a-963189aa7854/80019316/ATL03_20200427193622_04930702_006_02_subsetted.h5']

We need AWS credentials to access the S3 bucket with the results.  These are returned using the `aws_credentials` method.

_what about if we use our own bucket?_
_AS: Andy, why don't we leave this guidance out for the first iterationa and add a new Issue as a future enhancement. I don't think this is necessary for our intial publication and it could take some time to learn and document in the tutorial. But agree it's a great addition to include (in the future)_ 

In [35]:
creds = harmony_client.aws_credentials()

We then create a virtual file system that allows us to access the S3 bucket.  We pass the credentials to authenticate.

In [36]:
s3_fs = s3fs.S3FileSystem(
    key=creds['aws_access_key_id'],
    secret=creds['aws_secret_access_key'],
    token=creds['aws_session_token'],
    client_kwargs={'region_name':'us-west-2'},
)

We then open the S3 url as a _file-like_ object.

:::{note}
A _file-like_ object is just what it sounds like, an _object_ - a collection of bytes in memory - that is recognized as a file by applications.
:::

In [37]:
f = [s3_fs.open(url, mode='rb') for url in urls]

We can then open one of the files using `xarray`. 

In [38]:
ds = xr.open_dataset(f[0], group='gt1l/heights')
ds


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/srv/conda/envs/notebook/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/srv/conda/envs/notebook/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/srv/conda/envs/notebook/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/srv/conda/envs/notebook/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
 

AttributeError: _ARRAY_API not found

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## Appendices

_AS Andy: We could also remove this entirely for the first iteration and just point to harmony-py docs above when you first reference the 300 limit. This could be a future enhancement to add more examples or use cases as an Appendix._

### A. Handling large requests

Need to explain large-jobs and previews, pausing

https://github.com/nasa/harmony-py/blob/main/examples/job_pause_resume.ipynb

By default Harmony will only process first 300 granules

https://harmony.earthdata.nasa.gov/docs#skipping-preview

In [23]:
request = Request(
  collection = Collection(id=capabilities["conceptId"]),
  spatial=BBox(-105.5,40,-105,41.),
  # temporal={
  #   'start': dt.datetime(2020, 4, 27),
  #   'stop': dt.datetime(2020, 5, 28)
  # }
)
job_id = harmony_client.submit(request)
harmony_client.status(job_id)

{'status': 'running',
 'message': 'The job is being processed',
 'progress': 0,
 'created_at': datetime.datetime(2024, 10, 29, 22, 14, 21, 784000, tzinfo=tzlocal()),
 'updated_at': datetime.datetime(2024, 10, 29, 22, 14, 21, 784000, tzinfo=tzlocal()),
 'created_at_local': '2024-10-29T22:14:21+00:00',
 'updated_at_local': '2024-10-29T22:14:21+00:00',
 'request': 'https://harmony.earthdata.nasa.gov/C2596864127-NSIDC_CPRD/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?forceAsync=true&subset=lat(40%3A41.0)&subset=lon(-105.5%3A-105)',
 'num_input_granules': 134,
 'data_expiration': datetime.datetime(2024, 11, 28, 22, 14, 21, 784000, tzinfo=tzlocal()),
 'data_expiration_local': '2024-11-28T22:14:21+00:00'}

In [24]:
harmony_client.wait_for_processing(job_id, show_progress=True)

 [ Processing:  80% ] |########################################           | [/]
Job is running with errors.
 [ Processing: 100% ] |###################################################| [|]


In [25]:
harmony_client.status(job_id)

{'status': 'complete_with_errors',
 'message': 'The job has completed with errors. See the errors field for more details',
 'progress': 100,
 'created_at': datetime.datetime(2024, 10, 29, 22, 14, 21, 784000, tzinfo=tzlocal()),
 'updated_at': datetime.datetime(2024, 10, 29, 22, 29, 32, 444000, tzinfo=tzlocal()),
 'created_at_local': '2024-10-29T22:14:21+00:00',
 'updated_at_local': '2024-10-29T22:29:32+00:00',
 'request': 'https://harmony.earthdata.nasa.gov/C2596864127-NSIDC_CPRD/ogc-api-coverages/1.0.0/collections/all/coverage/rangeset?forceAsync=true&subset=lat(40%3A41.0)&subset=lon(-105.5%3A-105)',
 'num_input_granules': 134,
 'data_expiration': datetime.datetime(2024, 11, 28, 22, 14, 21, 784000, tzinfo=tzlocal()),
 'data_expiration_local': '2024-11-28T22:14:21+00:00',
 'errors': [{'url': 'https://data.nsidc.earthdatacloud.nasa.gov/nsidc-cumulus-prod-protected/ATLAS/ATL03/006/2023/08/20/ATL03_20230820094935_09352002_006_02.h5',
   'message': 'WorkItem failed: sds/trajectory-subsetter

### B. Translating requests

`harmony-py` is an interface to the Harmony RESTful API.  RESTful API send requests and receive responses via HTTPS; the same protocol that serves web pages.  Requests are sent to service endpoints, which is a URL (e.g. `https://service.endpoint.com/type_of_service`).  Query parameters that modify a request can be appended a key-value pairs after a `?`.  Each key-value pair is separated by an &.  For example:

```
https://service.endpoint.com/type_of_service?param1=value1&param2=value2
```

This url could be entered into a web browser.

The Harmony client has a `request_as_url` method that returns the request created by `CapabilitiesRequest` or `Request`

In [10]:
harmony_client.request_as_url(capabilities_request)

'https://harmony.uat.earthdata.nasa.gov/capabilities?shortname=ATL03'

In [11]:
harmony_client.request_as_url(request)

'https://harmony.uat.earthdata.nasa.gov/C1256407609-NSIDC_CUAT/ogc-api-coverages/1.0.0/collections/parameter_vars/coverage/rangeset?forceAsync=true&subset=lat%2840%3A41.0%29&subset=lon%28-105.5%3A-105%29&subset=time%28%222020-04-27T00%3A00%3A00%22%3A%222020-05-28T00%3A00%3A00%22%29&variable=all'

Try pasting the URL in the output cell below into a web browser to see the response.

Harmony requests can also be sent with `curl`, a library and command line tool for transfering data using various network protocols, including HTTPS.  The `request_as_curl` method can be used to generate a curl command.  This can be helpful if you want to automate a process outside of a Jupyter Notebook or for testing requests.

In [12]:
harmony_client.request_as_curl(capabilities_request)

"curl -X GET -H 'Accept: */*' -H 'Accept-Encoding: gzip, deflate, br' -H 'Authorization: *****' -H 'Connection: keep-alive' -H 'Cookie: uat_urs_user_already_logged=yes; state=s%3A7893b9a533c0a171525bc82a3693d9b5.XXQF2YV1GMxMt5SI%2BEQy05yxJC8KLMzUQRbpfAi1MqY; token=*****; _urs_uat-gui_session=fb95333993ef92e65ff1fe8efd4e7449' -H 'User-Agent: CPython/3.10.14 Linux/5.10.224-212.876.amzn2.x86_64 harmony-py/0.5.0 python-requests/2.32.3' 'https://harmony.uat.earthdata.nasa.gov/capabilities?shortname=ATL03'"

In [13]:
harmony_client.request_as_curl(request)

'curl -X POST -H \'Accept: */*\' -H \'Accept-Encoding: gzip, deflate, br\' -H \'Authorization: *****\' -H \'Connection: keep-alive\' -H \'Content-Length: 563\' -H \'Content-Type: multipart/form-data; boundary=fa4d368a0e27040358124f0e030b456b\' -H \'Cookie: uat_urs_user_already_logged=yes; state=s%3A7893b9a533c0a171525bc82a3693d9b5.XXQF2YV1GMxMt5SI%2BEQy05yxJC8KLMzUQRbpfAi1MqY; token=*****; _urs_uat-gui_session=fb95333993ef92e65ff1fe8efd4e7449\' -H \'User-Agent: CPython/3.10.14 Linux/5.10.224-212.876.amzn2.x86_64 harmony-py/0.5.0 python-requests/2.32.3\' -d \'--fa4d368a0e27040358124f0e030b456b\r\nContent-Disposition: form-data; name="forceAsync"\r\n\r\ntrue\r\n--fa4d368a0e27040358124f0e030b456b\r\nContent-Disposition: form-data; name="subset"\r\n\r\nlat(40:41.0)\r\n--fa4d368a0e27040358124f0e030b456b\r\nContent-Disposition: form-data; name="subset"\r\n\r\nlon(-105.5:-105)\r\n--fa4d368a0e27040358124f0e030b456b\r\nContent-Disposition: form-data; name="subset"\r\n\r\ntime("2020-04-27T00:00: